## GPT V3

单个模型(fixed), 直接对接到webui (生成很多个), 所以产生一行一个的结果

In [1]:
from tqdm.auto import tqdm
import gradio as gr
import transformers
import torch
import os


# Assuming DEVICE is already defined
DEVICE = 'cuda'  # Use 'cuda' for GPU or 'cpu' for CPU

# Path to the models' directories
MODEL_DIRS = [
    'pixiv-prompts-gpt-finetunes/8xh100_run2_e2_s50k',
]

# Load models and tokenizers from directories
models = {}
tokenizers = {}
for model_dir in MODEL_DIRS:
    model_name = os.path.basename(os.path.normpath(model_dir))
    models[model_name] = transformers.GPT2LMHeadModel.from_pretrained(model_dir).to(DEVICE)
    tokenizers[model_name] = transformers.AutoTokenizer.from_pretrained(model_dir)

def generate_text(rating: str, date: str, quality: str, character: str, prompt: str, max_length: int, num_lines: int, selected_models: list):
    """Generate text based on input prompt for selected models, managing multiple lines output."""
    query_prompt = f'<input rating="{rating}" chara="{character}" date="{date}" quality="{quality}" tags="{prompt}">'
    query_prompt += "<output>"

    all_outputs = []
    for model_name in selected_models:
        model = models[model_name].eval()  # Set the model to evaluation mode
        tokenizer = tokenizers[model_name]
        for _ in tqdm(range(num_lines)):
            input_ids = tokenizer.encode(query_prompt, return_tensors='pt').to(DEVICE)
            output_sequences = model.generate(
                input_ids=input_ids,
                max_length=max_length + len(input_ids[0]),
                temperature=1.0,
                top_k=50,
                top_p=0.95,
                repetition_penalty=1.0,
                do_sample=True,
                num_return_sequences=1,
            )

            generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)

            # Find and remove the initial part up to <output>
            start_output = generated_text.find("<output>") + len("<output>")
            generated_text = generated_text[start_output:].strip()

            # Remove the ending </output> tag or truncate at last complete tag
            end_tag = generated_text.find("</output>")
            if end_tag != -1:
                generated_text = generated_text[:end_tag]
            else:
                last_comma = generated_text.rfind(",")
                if last_comma != -1:
                    generated_text = generated_text[:last_comma]

            all_outputs.append(generated_text.strip())

    # Join all outputs into a single string separated by new lines
    return "\n".join(all_outputs)

# Define Gradio interface components
checkbox_choices = [os.path.basename(os.path.normpath(model_dir)) for model_dir in MODEL_DIRS]
iface = gr.Interface(
    fn=generate_text,
    inputs=[
        gr.Radio(choices=["general", "nsfw"], label="Rating", value="safe"),
        gr.Radio(choices=["2000s", "2010s", "2020s"], label="Date", value="2020s"),
        gr.Radio(choices=["normal", "good", "excellent"], label="Quality", value="excellent"),
        gr.Textbox(lines=1, placeholder="Enter your prompt here...", label="Character"),
        gr.Textbox(lines=2, placeholder="Enter your prompt here...", label="Prompt"),
        gr.Slider(minimum=40, maximum=300, value=180, step=10, label="Max Length"),
        gr.Slider(minimum=1, maximum=20, value=1, step=1, label="Number of Lines"),
        gr.CheckboxGroup(choices=checkbox_choices, label="Select Models", value=checkbox_choices),
    ],
    outputs=gr.Textbox(label="Generated Texts"),
    title="GPT-2 Text Generation | Multi-Model Support",
    description="This interface generates text based on the input prompt using selected models. Each model is fine-tuned from GPT-2."
)

# Launch the interface
iface.launch(share=True)

IMPORTANT: You are using gradio version 4.21.0, however version 4.29.0 is available, please upgrade.
--------
Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://aac5bc502401460215.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [2]:
!pip install -U pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.9/407.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 66.3 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.16.3
    Uninstalling pydantic_core-2.16.3:
      Successfully uninstalled pydantic_core-2.16.3
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.14
    Uninstalling pydantic-1.10.14:
      Successfully uninstalled pydantic-1.10.14
